In [1]:
import numpy as np
import random

In [79]:
class Network():
    def __init__(self,sizes):
        """The list ``sizes`` contains the number of neurons in the
        respective layers of the network.  
        For example, if the list
        was [2, 3, 1] then it would be a three-layer network, 
        with the first layer containing 2 neurons, 
        the second layer 3 neurons,
        and the third layer 1 neuron.  
        The biases and weights for the network are initialized randomly, 
        using a Gaussian distribution with mean 0, and variance 1.  
        Note that the first layer is assumed to be an input layer, 
        and by convention we won't set any biases for those neurons,
        since biases are only ever used in computing the outputs from later layers."""
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y,1)for y in sizes[1:]]
        self.weights = [np.random.randn(y, x) 
                        for x, y in zip(sizes[:-1], sizes[1:])]
        
    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.biases, self.weights):
            a = rectilinear_vec(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The ``training_data`` is a list of tuples
        ``(x, y)`` representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If ``test_data`` is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""
                
        if test_data: n_test = len(test_data)
        n = len(training_data)
        for j in range(epochs):
                        
            print (self.weights)
                        
                        
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print ("Epoch {0}: {1} / {2}".format(
                    j, self.evaluate(test_data), n_test))
            else:
                print( "Epoch {0} complete".format(j))

    def update_mini_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
        is the learning rate."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw 
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb 
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = softplus_vec(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            softplus_prime_vec(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in range(2, self.num_layers):
            z = zs[-l]
            spv = softplus_prime_vec(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * spv
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
#        print self.feedforward(test_data[0][0])
#        print self.feedforward(test_data[1][0])
        test_results = [(np.argmax(self.feedforward(x)), np.argmax(y)) 
                        for (x, y) in test_data]
#        print test_results
        return sum(int(x == y) for (x, y) in test_results)
        
    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y) 

#### Miscellaneous functions
def rectilinear(z):
    return max(z, 0)
#    return 1.0/(1.0+np.exp(-z))

rectilinear_vec = np.vectorize(rectilinear)

def rectilinear_prime(z):
    return 1*(z>=0)
#    return sigmoid(z)*(1-sigmoid(z))



rectilinear_prime_vec = np.vectorize(rectilinear_prime)

def softplus(z):
    return np.log(1+np.exp(5*z))/5
   #return 1.0/(1.0+np.exp(-z))

softplus_vec = np.vectorize(softplus)

def softplus_prime(z):
    return np.exp(5*z)/(np.exp(5*z)+1)

softplus_prime_vec = np.vectorize(softplus_prime)

#traing_data, validation_data, test_data = mnist_loader.load_wrapper()

training_data = ([(np.array([[-1], [-1]]), np.array([[1], [0]]))]*100) + \
                ([(np.array([[-1], [1]]), np.array([[0], [1]]))]*100) + \
                ([(np.array([[1], [-1]]), np.array([[0], [1]]))]*100) + \
                ([(np.array([[1], [1]]), np.array([[1], [0]]))]*100)

test_data = ([(np.array([[-1], [-1]]), np.array([[1], [0]]))]*100) + \
                ([(np.array([[-1], [1]]), np.array([[0], [1]]))]*100) + \
                ([(np.array([[1], [-1]]), np.array([[0], [1]]))]*100) + \
                ([(np.array([[1], [1]]), np.array([[1], [0]]))]*100)                
random.shuffle(training_data)
#random.shuffle(validation_data)
random.shuffle(test_data)                
                
net =Network([2, 3, 2])

#print training_data
net.SGD(training_data, 200, 10, 0.04, test_data=test_data)


[array([[ 1.4789565 ,  1.05857989],
       [ 1.06774984,  0.76645683],
       [ 1.52369173, -0.80709457]]), array([[-1.47093048,  0.91717483, -1.99041459],
       [ 0.98985751, -0.15794871,  1.37832271]])]
Epoch 0: 200 / 400
[array([[ 0.89837227,  1.36496667],
       [ 1.32791237,  0.63850062],
       [ 1.09631722, -0.88702934]]), array([[-1.55262299,  0.88985885, -1.99782346],
       [ 0.53051066, -0.69484528,  0.80832818]])]
Epoch 1: 200 / 400
[array([[ 0.78085325,  1.43371829],
       [ 1.37394854,  0.65582863],
       [ 1.08602229, -0.918384  ]]), array([[-1.55574848,  0.88994706, -1.99907198],
       [ 0.59016226, -0.77687007,  0.81244135]])]
Epoch 2: 200 / 400
[array([[ 0.71037562,  1.46522942],
       [ 1.39829376,  0.68244639],
       [ 1.09526569, -0.93891189]]), array([[-1.55543451,  0.89036489, -2.00009206],
       [ 0.61969224, -0.83066717,  0.83210334]])]
Epoch 3: 200 / 400
[array([[ 0.66734855,  1.48359423],
       [ 1.41054218,  0.70258436],
       [ 1.10829078, -0.95057

In [65]:
sizes = [2,2,3]
for y in sizes[1:]:
    print (y)

2
3


In [30]:
np.random.randn(3, 5)

array([[-1.16233924, -0.56334343, -1.16397759,  0.81053974, -0.31435514],
       [-1.12331458,  1.67133084,  0.43740059,  1.21752052, -1.72481343],
       [ 0.05564476,  1.33750629,  0.5468868 , -0.43804514,  0.55548133]])

SyntaxError: invalid syntax (<ipython-input-7-e52cf5b99913>, line 1)

In [39]:
net = Network([2, 3, 3])

In [40]:
net.weights

[array([[ 0.95513699, -0.88033069],
        [-0.30129804, -1.18517362],
        [ 0.59145892, -0.44265403]]),
 array([[ 0.6843062 , -1.08916424, -0.49402575],
        [ 1.0230353 ,  0.97454081, -0.34096677],
        [-0.91941726, -0.21521995, -0.72673843]])]